In [3]:
import os
os.getcwd()

'/mnt/data/bio/Vincent/Sample Report Demo Data'

In [5]:
import ihapi
ihapi.login()

import pandas
import numpy
import os
import shutil

Loaded session from /home/vincent/.config/ihapi/session.pickle


# Inputs:

- participant_id
- cell_types
- abundance method (e.g. TPM)

# Outputs:

- per cell type:
  - volcano csv that is the output of deseq2_differential_expression
  - named: `{participant_id}/Volcanoes/{cell_type_name}.csv`
  
- per gene:
  - gene abundance csv with the columns: is_self, cell_type, abundance
  - named: `{participant_id}/Abundances/{gene_name}_.csv`

# Setup

In [6]:
#PARTICIPANT_ID = 353 # Vincent
PARTICIPANT_ID = 3 # David
ABUNDANCE_METHOD = "tpm"
CELL_TYPES = ["Monocytes", "NK Cells", "T Cells", "B Cells"]
HIERARCHY_LEVEL = 2
HIERARCHY_NAME = "Filtered WBCs 2023-03-21"
NUM_GENES = 5000
CHROMOSOME_SPECIFIC_GENE = "XIST"
# Determined via threshold_otsu(numpy.array(chromosome_specific_gene_abundances)) (and just looking at it)
CHROMOSOME_GENE_ABUNDANCE_THRESHOLD = 5e-5
DATA_DIR = "data"

In [7]:
data_directory = os.path.join(DATA_DIR, str(PARTICIPANT_ID))
abundances_directory = os.path.join(DATA_DIR, str(PARTICIPANT_ID), "Abundances")
volcanoes_directory = os.path.join(DATA_DIR, str(PARTICIPANT_ID), "Volcanoes")

In [8]:
for directory in [data_directory, abundances_directory, volcanoes_directory]:

    if os.path.exists(directory):
        shutil.rmtree(directory)
    
    os.makedirs(directory)

In [9]:
SAMPLE_PROCESSING_QUERY = [
    [
        ('TAP II Heparin',),
        ('Cryopreserving PBMCs',)
    ]
]

PARTICIPANT_QUERY = None

SINGLE_CELL_ASSAY_QUERY = [
    {
        "protocol_name": "10X v3.1 3 Prime"
    },
    {
        "protocol_name": "10X v3 3 Prime"
    },
    {
        "protocol_name": "10X v3.1 3 Prime HT"
    },
    {
        "protocol_name": "10X v3.1 LT"
    }
]

COMPUTATIONAL_PIPELINE = 5

In [10]:
#adata = ihapi.matrix.get_matrix(
#    sample_processing_workflows=SAMPLE_PROCESSING_QUERY,
#    participant=PARTICIPANT_QUERY,
#    cell_barcoding_run=SINGLE_CELL_ASSAY_QUERY,
#    process_chain_id=COMPUTATIONAL_PIPELINE
#)

In [13]:
import os
import pickle

def get_data():
    pickle_file = 'matrix.pickle'

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as file:
            adata = pickle.load(file)
    else:
        # Make a dummy API call
        adata = ihapi.matrix.get_matrix(
            sample_processing_workflows=SAMPLE_PROCESSING_QUERY,
            participant=PARTICIPANT_QUERY,
            cell_barcoding_run=SINGLE_CELL_ASSAY_QUERY,
            process_chain_id=COMPUTATIONAL_PIPELINE
        )
        ihapi.metadata.retrieve(adata, process_chain_id=COMPUTATIONAL_PIPELINE, funcs=[ihapi.metadata.funcs.batch])
        ihapi.annotate.apply_annotations(adata, HIERARCHY_NAME)

        # Save the data to the pickle file
        with open(pickle_file, 'wb') as file:
            print('Writing adata pickle...')
            pickle.dump(adata, file)

    return adata

def delete_pickle_file():
    pickle_file = 'matrix.pickle'

    if os.path.exists(pickle_file):
        os.remove(pickle_file)
        print(f"Deleted {pickle_file} successfully.")
    else:
        print(f"{pickle_file} does not exist.")
        
delete_pickle_file()
adata = get_data()

Deleted matrix.pickle successfully.
Multiple valid cell set feature matrices, returning first


Retrieved: 100%|██████████| 12.4G/12.4G [00:16<00:00, 797MB/s]


# Restrict analysis to only people in same chromosome parity equivalence class

In [14]:
below_threshold_participant_ids = set()
above_threshold_participant_ids = set()
is_participant_below_threshold = None
participant_gene_data = []

for participant_id in adata.obs["Participant IDs"].unique():
    
    participant_adata = adata[adata.obs["Participant IDs"] == participant_id]
    
    total_transcript_counts = participant_adata.X.sum()
    
    participant_gene_data.append(numpy.array(participant_adata.X.sum(axis=0)).flatten()/total_transcript_counts)
    
    chromosome_specific_gene_count = participant_adata[:, adata.var["name"] == CHROMOSOME_SPECIFIC_GENE].X.sum()
    chromosome_specific_gene_abundance = chromosome_specific_gene_count/total_transcript_counts
    
    if chromosome_specific_gene_abundance < CHROMOSOME_GENE_ABUNDANCE_THRESHOLD:
        below_threshold_participant_ids.add(participant_id)
        
        if participant_id == PARTICIPANT_ID:
            is_participant_below_threshold = True
    else:
        above_threshold_participant_ids.add(participant_id)
        
        if participant_id == PARTICIPANT_ID:
            is_participant_below_threshold = False
            
if is_participant_below_threshold:
    eligible_participant_ids = below_threshold_participant_ids
else:
    eligible_participant_ids = above_threshold_participant_ids
    
filtered_adata = adata[adata.obs["Participant IDs"].isin(eligible_participant_ids)].copy()

participant_gene_data = numpy.array(participant_gene_data)

In [15]:
gene_maxes = participant_gene_data.max(axis=0)
gene_mask = numpy.zeros(gene_maxes.shape, dtype=bool)
gene_mask[gene_maxes.argsort()[-NUM_GENES:]] = 1
filtered_adata = filtered_adata[:, gene_mask].copy()

In [16]:
filtered_adata.obs["is_self"] = filtered_adata.obs["Participant IDs"] == PARTICIPANT_ID
filtered_adata.obs["is_self"] = filtered_adata.obs["is_self"].astype(str)
filtered_adata.obs["batch"] = [f"B{batch}" for batch in filtered_adata.obs["batch"].astype(str)]

In [17]:
gene_abundances_data = []

for cell_type in CELL_TYPES:
    
    cell_type_adata = filtered_adata[filtered_adata.obs[f"cell_type_level_{HIERARCHY_LEVEL}"] == cell_type]
    
    for sample_id in cell_type_adata.obs["Participant IDs"].unique():
        
        cell_type_participant_adata = cell_type_adata[cell_type_adata.obs["Participant IDs"] == participant_id]
        total_transcript_counts = cell_type_participant_adata.X.sum()
        gene_abundances = numpy.array(cell_type_participant_adata.X.sum(axis=0)).flatten()
        tpms = gene_abundances / total_transcript_counts * 1e6
        
        gene_abundances_data.extend([
            {
                "is_self": str(participant_id == PARTICIPANT_ID),
                "abundance_value": tpm,
                "cell_type": cell_type,
                "gene_name": gene_name
            }
            
            for tpm, gene_name in zip(tpms, cell_type_participant_adata.var["name"].values)
        ])
    
    de_df = ihapi.differential_expression.deseq2_differential_expression(
        cell_type_adata,
        formula='~is_self',
        sample_column='Sample IDs',
        contrast_elements=['is_self', 'True', 'False']
    )
    
    file_name = os.path.join(volcanoes_directory, f"{cell_type}.csv")
    
    de_df["gene_name"] = filtered_adata.var.loc[de_df.index.values.astype(str)]["name"].values
    de_df["magstat"] = numpy.abs(de_df["stat"])
    de_df.dropna(inplace=True)
    de_df.rename(columns={ "-log10padj": "log10padj" }, inplace=True)
    
    de_df.to_csv(file_name)

with open(os.path.join(data_directory, "gene_list.csv"), "w") as genes_file:
    genes_file.write("\n".join(filtered_adata.var.loc[de_df.index.values.astype(str)]["name"].values))

In [18]:
try:
    !rm *.log
except:
    pass
try:
    !rm *.csv
except:
    pass
try:
    !rm *.R
except:
    pass

rm: cannot remove '*.csv': No such file or directory
rm: cannot remove '*.R': No such file or directory


In [19]:
gene_abundances_df = pandas.DataFrame.from_records(gene_abundances_data)

for gene_name in gene_abundances_df["gene_name"].unique():
    
    gene_df = gene_abundances_df[gene_abundances_df["gene_name"] == gene_name].copy()
    
    gene_df.drop("gene_name", axis=1, inplace=True)
    gene_df.dropna(inplace=True)
    
    file_name = os.path.join(abundances_directory, f"{gene_name}.csv")
    
    gene_df.to_csv(file_name)
    
    if gene_df["abundance_value"].isna().sum() > 0:
        print(gene_name)

In [20]:
import zipfile
import os

def zip_directory(directory_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for folder_name, subfolders, file_names in os.walk(directory_path):
            for file_name in file_names:
                file_path = os.path.join(folder_name, file_name)
                zip_file.write(file_path, arcname=os.path.relpath(file_path, directory_path))

zip_directory(data_directory, os.path.join("data", f"{PARTICIPANT_ID}.zip"))